In [ ]:
# Training dataset
import pandas as pd
import os
from data_engineering import create_timeseries_for_symbol, prepare_regression_data, clean_data
from pca import reduce_dimensions_pca

if __name__ == "__main__":

    output_dir = "symbol_data"
    symbol_id = 0
    csv_path = os.path.join(output_dir, f"symbol_{symbol_id}.csv")
    df = pd.read_csv(csv_path)

    features, responders, target = create_timeseries_for_symbol(df, symbol_id)
    clean_features, clean_responders = clean_data(features, responders)

    X, y = prepare_regression_data(clean_features, clean_responders, target=target)

    # For specific number of components:
    X_reduced, pca, scaler = reduce_dimensions_pca(X, n_components=25)


In [ ]:
# Test dataset
from data_engineering import clean_data

if __name__ == "__main__":
    features = pd.read_parquet("test.parquet/date_id=0/part-0.parquet")
    responders = pd.read_parquet("lags.parquet/date_id=0/part-0.parquet")

    symbol_features = features[features['symbol_id'] == symbol_id].copy()

    symbol_responders = responders[responders['symbol_id'] == symbol_id].copy()

    # Select feature columns
    feature_cols = [col for col in symbol_features.columns if 'feature_' in col]

    # Select responder columns except responder_6
    responder_cols = [col for col in symbol_responders.columns if 'responder_' in col and 'responder_6' not in col]

    # Create combined dataset
    features_to_use = symbol_features[feature_cols]
    responders_to_use = symbol_responders[responder_cols]

    clean_features, clean_responders = clean_data(features_to_use, responders_to_use)

    X = prepare_regression_data(clean_features, clean_responders)

    # For specific number of components:
    X_reduced, pca, scaler = reduce_dimensions_pca(X, n_components=25)
